In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Volume Estimation and Uncertainty Analysis

### 2D Volume Estimation

In petrophysics, 2D volume estimation is used to calculate the volume of hydrocarbons in a reservoir using two-dimensional data, typically from seismic surveys or well logs. Here's a simplified process:

1. **Data Collection**: Gather 2D seismic data and well log data. Seismic data provides a cross-sectional view of the subsurface, while well logs give detailed information about the rock and fluid properties at specific locations.

2. **Structural Mapping**: Create structural maps of the reservoir. This involves interpreting seismic data to delineate the boundaries and structure of the reservoir.

3. **Reservoir Zoning**: Divide the reservoir into zones based on geological and petrophysical properties. Each zone may have different characteristics affecting hydrocarbon volume.

4. **Property Assignment**: Assign petrophysical properties (e.g., porosity, water saturation) to each zone. These properties are derived from well logs and core samples.

5. **Volume Calculation**: Calculate the volume of each zone using the formula:
   $$
   V = A \times h \times \phi \times (1 - S_w)
   $$
   where:
   - \( V \) is the hydrocarbon volume.
   - \( A \) is the area of the zone.
   - \( h \) is the thickness of the zone.
   - \( \phi \) is the porosity.
   - \( S_w \) is the water saturation.

### Uncertainty Analysis

Uncertainty analysis is crucial because it helps quantify the confidence in the volume estimates. Here are the key steps:

1. **Identify Uncertainties**: Determine the sources of uncertainty, such as data quality, interpretation errors, and variability in petrophysical properties.

2. **Quantify Uncertainties**: Use statistical methods to quantify the uncertainties. This can involve:
   - **Monte Carlo Simulation**: Generate multiple realizations of the reservoir model by varying the input parameters within their uncertainty ranges.
   - **Sensitivity Analysis**: Assess how changes in each parameter affect the volume estimate.

3. **Probabilistic Volume Estimation**: Combine the results from the simulations to create a probabilistic distribution of the hydrocarbon volume. This provides a range of possible volumes with associated probabilities.

4. **Risk Assessment**: Evaluate the risk associated with different volume estimates. This helps in making informed decisions about reservoir development and management.

quick_pp estimates the volume by considering the distribution of each parameter. Smaller range and smaller standard deviation indicates less uncertainty and vice versa applies.

Monte Carlo simulation is implemented ...

In [ ]:
import pandas as pd

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
df = project.get_all_data()

In [ ]:
from quick_pp.plotter import stick_plot

# Define well configuration
well_config = {
    'HW-10': {'sorting': 1, 'OUT': 8400, 'ODT': 8550},
    'HW-24': {'sorting': 2, 'GUT': 8300, 'GOC': 8300, 'ODT': 8590},
    'HW-25': {'sorting': 4, 'GOC': 8300, 'ODT': 8410, 'GUT': 8210},
    'HW-26': {'sorting': 3, 'OUT': 8400, 'OWC': 8600},
    'HW-27': {'sorting': 6, 'WUT': 8600},
    'HW-28': {'sorting': 5, 'WUT': 8600},
    'HW-29': {'sorting': 7, 'GUT': 8100, 'GDT': 8400},
    'HW-30': {'sorting': 9, 'GUT': 8300, 'GOC': 8300, 'ODT': 8600},
    'HW-31': {'sorting': 8, 'WUT': 8600},
    'HW-32': {'sorting': 11, 'OUT': 8300, 'ODT': 8600},
    'HW-4': {'sorting': 10, 'GUT': 8100, 'GDT': 8400},
    'HW-5': {'sorting': 13, 'GUT': 8500, 'GDT': 8600},
    'HW-6': {'sorting': 12, 'GUT': 7900, 'GDT': 8150},
    'HW-7': {'sorting': 15, 'WUT': 8600},
    'HW-8': {'sorting': 16, 'WUT': 8600},
    'HW-9': {'sorting': 14, 'WUT': 8600}
}

# Plot stick plot
stick_plot(df, well_config)

In [ ]:
# Convert well configuration dictionary to pandas DataFrame
well_config_df = pd.DataFrame.from_dict(well_config, orient='index')

# Batch update on the contacts
conf_df = well_config_df.copy()
conf_df['GUT'] = 7900
conf_df['GDT'] = conf_df['GOC'] = conf_df['OUT'] = 8300
conf_df['OWC'] = conf_df['ODT'] = conf_df['WUT'] = 8600
conf_dict = conf_df.to_dict(orient='index')

stick_plot(df, conf_dict)

In [ ]:
from quick_pp.ressum import mc_volumetric_method

n_try = 10000
reserves = mc_volumetric_method(
    area_bound=(120, 840, 660, 160),
    thickness_bound=(3, 9, 6, 3), 
    porosity_bound=(.2, .3, .1, .05),
    water_saturation_bound=(.2, .35, .3),
    volume_factor_bound=(1.2, 1.4),
    recovery_factor_bound=(.1, .5, .3, .075),
    n_try=n_try, percentile=[5, 30, 50, 70, 95]
)
